## Genrate the CSV file which contains all the images file_name, player_name, gender, bounding_box

In [4]:
import os
import pandas as pd
import json
import shutil
import random
import numpy as np

In [5]:
def generate_csv(img_path,json_path,out_path):
    
    ## from the image folder 1st create players names
    country = img_path.split('/')[-1].split('_')[0]
    print(country)
    files_path = img_path+"/normal"
    file_names = os.listdir(files_path)
    df = pd.DataFrame()
    df['filename'] = None
    df['cricketer_name'] = None
    df['gender'] = None
    df['bounding_box'] = None
    
    for file in file_names:
        file_path = files_path+"/"+file
        filename = file.split('.')[0]
        cricketer_name = file.split('.')[0].split('_')[0]
        df.loc[len(df.index)] = [filename,cricketer_name,None,None]
    
    ## Getting the gender from the JSON file
    df_json = pd.DataFrame()
    df_json['filename'] = None
    df_json['gender'] = None
    json_files = os.listdir(json_path)
    for file in json_files:
        file_path = json_path+'/'+file
        filename = file.split('.')[0]
        f = open(file_path)
        data = json.load(f)
        gender = gender = data['gender']
        f.close()
        df_json.loc[len(df_json.index)] = [filename,gender]
    
    ## Matching image file with JSON file and keep the gender only
    for i in range(len(df)):
        filename = df['filename'][i]
        row_index = df_json[df_json['filename'] == filename].index.item()
        df.at[i,'gender'] = df_json['gender'][row_index]
    print(len(df))
    ## Drop row where 'gender' column in None
    df = df.dropna(subset=['gender'])
#     print(len(df))
    df.to_csv(out_path+country+".csv",index = False)
#     print(df)

In [6]:
img_path_all = "./../All Face Images with And Without Mask (N95)"
json_path_all = "./../Details"

img_path_all_lis = os.listdir(img_path_all)
json_path_all_lis = os.listdir(json_path_all)
for img_path,json_path in zip(img_path_all_lis,json_path_all_lis):
    img_path = img_path_all+'/'+img_path
    json_path = json_path_all+'/'+json_path
#     generate_csv(img_path,json_path,"./../image_details/")

## Get the number of male and female images for each country

In [7]:
def get_male_female_count(path):
    files_csv = os.listdir(path)
    for file in files_csv:
        country = file.split('.')[0]
        file_path = path+"/"+file
        print("####### "+country+" #######")
        df = pd.read_csv(file_path)
        total = len(df)
        male = len(df[df['gender'] == 'M'])
        female = len(df[df['gender'] == 'F'])
        print("Total : ",total)
        print("Male : ",male)
        print("Female : ",female)

get_male_female_count("./../image_details")

####### Australia #######
Total :  941
Male :  760
Female :  181
####### Bangladesh #######
Total :  367
Male :  328
Female :  39
####### England #######
Total :  1418
Male :  1285
Female :  133
####### India #######
Total :  2254
Male :  2066
Female :  188
####### New Zealand #######
Total :  597
Male :  365
Female :  232
####### Pakistan #######
Total :  557
Male :  478
Female :  79
####### South Africa #######
Total :  541
Male :  482
Female :  59
####### West Indies #######
Total :  649
Male :  584
Female :  65


## Randomly Separate same number of male and female images 

In [8]:
def create_folder(path):
    try:
        os.mkdir(path)
    except:
        pass

In [33]:
def save_to_folder(df_final,df,indices,img_path_country,out_img_path_train,gender):
    out_gender = "male"
    if gender == 'F':
        out_gender = "female"
    for i in indices:
        df_final.loc[len(df_final.index)] = [df['filename'][i],gender]
        filename = df['filename'][i]+'.jpg'
        if img_type == "masked":
            filename = df['filename'][i]+'_N95.jpg'
        src = img_path_country+"/"+filename
        dst = out_img_path_train+'/'+out_gender+'/'+filename
        shutil.copy(src,dst)

def separate_images(img_details_path,img_path,out_img_path,colored_images,out_details_path,cnt,skintone_csv,img_type):
    countries = os.listdir(img_details_path)
    df_final = pd.DataFrame()
    df_final['filename'] = None
    df_final['gender'] = None
    for country in countries:
        country = country.split('.')[0]
        print(country)
        img_details_path_country = img_details_path+"/"+country
        img_path_country = img_path+"/"+country
#         img_path_country = img_path+"/"+country+"_resized/normal"
        colored_images_country = colored_images+"/"+country
    
        #### IF img_type == MASKED then there are not all images
        df_masked = pd.DataFrame()
        if img_type == 'masked':
            all_masked_images_ = os.listdir(img_path_country)
            all_masked_images = [filename.replace('_N95.jpg', '') for filename in all_masked_images_]
            df_masked['filename'] = all_masked_images
            
        df = pd.read_csv(img_details_path_country+'.csv',header = 0)
        df_color = pd.read_csv(colored_images_country+'.csv',header = 0)
        df_color = df_color[df_color['status'] == 'OK']
        ## Keep only images which are colored images
        df_color = df_color.reset_index(drop=True)
        for i in range(len(df_color)):
            df_color.at[i,'fileName'] = df_color['fileName'][i].split('.')[0]
        df_color = df_color.rename(columns={'fileName': 'filename'})
        ## Merge 2 dataframe
        df = pd.merge(df[['filename','gender']],df_color[['filename']],on = 'filename')
        
        ## Merge the dataframe with the valid masked images####
        if img_type == 'masked':
            df = pd.merge(df[['filename','gender']],df_masked[['filename']],on = 'filename')
        
        df_male = df[df['gender'] == 'M']
        df_male = df_male.reset_index(drop=True)
        df_female = df[df['gender'] == 'F']
        df_female = df_female.reset_index(drop=True)
        print("male :",len(df_male))
        print("female :",len(df_female))
        ############################### TRAINING DATA ####################################
        ## Create male and female folder (FOR TRAINING DATA)
        out_img_path_train = out_img_path+"/train"
        create_folder(out_img_path_train)
        create_folder(out_img_path_train+"/male")
        create_folder(out_img_path_train+"/female")
        
        
        
        ## ######### If country is WEST INDIES then we need to consider the skin-tone also #############
        if country == "South Africa" and cnt > 1:
            df_skin = pd.read_csv(skintone_csv+country+'.csv')
            df_skin = df_skin.sort_values(by='class')
            
            df_skin_fair = df_skin[df_skin['class'] <= "Monk 06"]
            df_skin_dark = df_skin[df_skin['class'] >= "Monk 07"]
            
            df_fair = pd.merge(df,df_skin_fair,left_on='filename',right_on='file',how='inner')
            df_dark = pd.merge(df,df_skin_dark,left_on='filename',right_on='file',how='inner')
            df_fair = df_fair.drop('file',axis = 1)
            df_dark = df_dark.drop('file',axis = 1)
            
            df_fair_male = df_fair[df_fair['gender'] == 'M']
            df_fair_male = df_fair_male.reset_index()
            df_dark_male = df_dark[df_dark['gender'] == 'M']
            df_dark_male = df_dark_male.reset_index()
            df_fair_female = df_fair[df_fair['gender'] == 'F']
            df_fair_female = df_fair_female.reset_index()
            df_dark_female = df_dark[df_dark['gender'] == 'F']
            df_dark_female = df_dark_female.reset_index()
            
#             print(df_fair_male)
#             print(df_dark_male)
#             print(df_fair_female)
#             print(df_dark_female)
            
            num_dark = cnt//2
            num_fair = cnt - num_dark
            
            ## Save the images in folder ##
            ## Fair Male
            fair_male_indices = random.sample(range(0, len(df_fair_male)), num_fair)
            save_to_folder(df_final,df_fair_male,fair_male_indices,img_path_country,out_img_path_train,'M')
            
            ## Dark Male
            dark_male_indices = random.sample(range(0, len(df_dark_male)), num_dark)
            save_to_folder(df_final,df_dark_male,dark_male_indices,img_path_country,out_img_path_train,'M')
            
            ## Fair Female
            fair_female_indices = random.sample(range(0, len(df_fair_female)), num_fair)
            save_to_folder(df_final,df_fair_female,fair_female_indices,img_path_country,out_img_path_train,'F')
            
            ## Dark Female
            dark_female_indices = random.sample(range(0, len(df_dark_female)), num_dark)
            save_to_folder(df_final,df_dark_female,dark_female_indices,img_path_country,out_img_path_train,'F')
            
            continue
        
        
        male_ind =  random.sample(range(0, len(df_male)), cnt)
        female_ind =  random.sample(range(0, len(df_female)), cnt)
        ## Pick cnt number of male and female images
        for i in male_ind:
            df_final.loc[len(df_final.index)] = [df_male['filename'][i],'M']
            
            filename = df_male['filename'][i]+'.jpg'
            if img_type == "masked":
                filename = df_male['filename'][i]+'_N95.jpg'
                
            src = img_path_country+"/"+filename
            dst = out_img_path_train+'/male/'+filename
            shutil.copy(src,dst)
            
        for i in female_ind:
            df_final.loc[len(df_final.index)] = [df_female['filename'][i],'F']
            filename = df_female['filename'][i]+'.jpg'
            if img_type == "masked":
                filename = df_female['filename'][i]+'_N95.jpg'
            src = img_path_country+"/"+filename
            dst = out_img_path_train+'/female/'+filename
            shutil.copy(src,dst)
        print(len(male_ind),len(female_ind))
        '''
        ############################### TEST DATA ####################################
        out_img_path_test = out_img_path+"/test"
        create_folder(out_img_path_test)
        create_folder(out_img_path_test+"/male")
        create_folder(out_img_path_test+"/female")
        # Select the indices not present in the list
#         male_ind_test = df_male[~df_male.index.isin(male_ind)].index
        male_ind_test = np.setdiff1d(df_male.index, male_ind)
#         female_ind_test = df_female[~df_female.index.isin(female_ind)].index
        female_ind_test = np.setdiff1d(df_female.index, female_ind)
#         print(list(female_ind_test))
        ## Copy male images
        for i in male_ind_test:
            filename = df_male['filename'][i]+'.jpg'
            src = img_path_country+"/"+filename
            dst = out_img_path_test+'/male/'+filename
            shutil.copy(src,dst)
        ## Copy female images
        for i in female_ind_test:
            filename = df_female['filename'][i]+'.jpg'
            src = img_path_country+"/"+filename
            dst = out_img_path_test+'/female/'+filename
            shutil.copy(src,dst)
        '''
    df_final.to_csv("details_"+img_type+"_"+str(cnt)+".csv",index = False)

In [36]:

count = 10
img_type = "masked"

img_details_path = "./../image_details"
img_path = "./../Players Dataset_v2/Masked"
colored_images = "./../correction/status_csv_files"
skintone_csv = "./../Skintone/skin_tone_csv/"
out_img_path = "data_"+img_type+"_"+str(count)

create_folder(out_img_path)
separate_images(img_details_path,img_path,out_img_path,colored_images,".",count,skintone_csv,img_type)

Australia
male : 651
female : 158
10 10
Bangladesh
male : 319
female : 39
10 10
England
male : 917
female : 111
10 10
India
male : 1966
female : 187
10 10
New Zealand
male : 312
female : 221
10 10
Pakistan
male : 435
female : 78
10 10
South Africa
male : 422
female : 58
West Indies
male : 488
female : 63
10 10
